In [1]:
import midi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import keras
import warnings
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")
%matplotlib inline

Using TensorFlow backend.


In [2]:
class MidiTokenizer():
    def __init__(self, filepath, padding=True):
        '''
        filepath: path to MIDI file
        padding: whether or not to pad time series data in case of polyphony
        self.mid: python-midi parsing of MIDI file
        self.format: format of MIDI file (0, 1, 2)
        self.tick_relative: whether event ticks are recorded relative to other
            events or absolutely
        '''
        self.mid = midi.read_midifile(filepath)
        self.format = self.mid.format
        self.tick_relative = self.mid.tick_relative
        self.padding = padding
        
        self.event_data = {i: [] for i in range(17)}
        self.time_series_data = {}
        
        if self.format == 0:         # one midi track
            self.parse_track([mid], self.tick_relative)
        elif self.format == 1:       # multiple midi tracks
            for track in self.mid:
                self.parse_track(track, self.tick_relative)
                
        for i in self.event_data.keys():
            self.event_data[i] = pd.DataFrame(
                self.event_data[i], columns=['start', 'duration', 'pitch', 'velocity'])
            
        self.gen_time_series_data()
                            
    def parse_track(self, track, tick_relative=True):        
        tick = 0
        pitch_data = {}
        
        for event in track:
            if type(event) != midi.NoteOnEvent and type(event) != midi.NoteOffEvent:
                # if event does not correspond to a note, skip event
                continue

            tick = (tick + event.tick) if tick_relative else event.tick
            
            if event.velocity != 0 and type(event) != midi.NoteOffEvent:
                # note on
                datum = [tick, None, event.pitch, event.velocity]
                self.event_data[event.channel].append(datum)
                try:
                    if pitch_data[event.pitch][-1][1] == None:
                        pitch_data[event.pitch][-1][1] = tick
                        
                    pitch_data[event.pitch].append(datum)
                except KeyError:
                    pitch_data[event.pitch] = [datum]
            else:
                # note off
                pitch_data[event.pitch][-1][1] = tick - self.event_data[event.channel][-1][0]
                                
    def gen_time_series_data(self):
        for channel, parsed_track in self.event_data.items():
            if parsed_track.shape[0] == 0:
                continue
            
            ticks = parsed_track[['start', 'duration']].values
            max_tick = max(map(lambda x:np.sum(x), ticks)) + 1
            
            self.time_series_data[channel] = [[] for _ in range(max_tick)]
            
            for _, event in parsed_track.iterrows():
                start, duration, pitch, velocity = event.values
                end = start + duration
                note = (pitch, velocity)
                                
                self.time_series_data[channel][start].append(note)
                self.time_series_data[channel][end].append(note)
                
            activated_notes = set([])
            
            for i, notes in enumerate(self.time_series_data[channel]):
                for note in notes:
                    if note not in activated_notes:
                        activated_notes.add(note)
                    else:
                        activated_notes.remove(note)
                
                self.time_series_data[channel][i] = list(activated_notes)
              
            if self.padding:
                self.time_series_data[channel] = \
                    keras.preprocessing.sequence.pad_sequences(self.time_series_data[channel])

            

In [3]:
mt = MidiTokenizer('data/no_quarter/NoQuarter.mid')

In [4]:
def process_data(data, timestep):
    X, y = [], []
    for i in range(len(data)-timestep-1):
        X.append(np.array([data[i:(i+timestep)]]))
        y.append(np.array([data[(i+timestep)]]))
        
    X, y = np.array(X), np.array(y)
    return X.reshape(*[_ for _ in X.shape if _ != 1]), \
           y.reshape(*[_ for _ in y.shape if _ != 1])

def cantor_pair(k1, k2):
    return int((k1+k2)*(k1+k2+1)/2+k2)

def cantor_unpair(z):
    w = int((np.sqrt(8*z+1)-1)/2)
    t = (w**2+w)/2
    y = int(z-t)
    x = int(w-y)
    return x, y

In [5]:
class RNN:
    def __init__(self, X, train_test_split=0.8, epochs=100, batch_size=10, lstm_units=128, timestep=78):
        self.X = X
        self.split = int(self.X.shape[0]*train_test_split)
        self.X_train, self.X_test = self.X[:self.split], self.X[self.split:]
        self.epochs = epochs
        self.batch_size = batch_size
        self.lstm_units = lstm_units
        self.timestep = timestep
        self.scaler = StandardScaler()
        self.scale_data()
        
        self.model = Sequential()
        self.model.add(
            LSTM(self.lstm_units, input_shape=(
                self.timestep, self.X_train_processed.shape[-1]
            ))
        )
        self.model.add(Dense(self.X_train_processed.shape[-1]))
        self.model.compile(optimizer='adamax', loss='mse')
        
    def flatten_data(self):
        self.X_train_flattened = np.apply_along_axis(lambda x:x[1], 2, self.X_train)
        self.X_test_flattened = np.apply_along_axis(lambda x:x[1], 2, self.X_test)
        
    def scale_data(self):
        self.flatten_data()
        self.scaler.fit(self.X_train_flattened)
        self.X_train_scaled = self.scaler.transform(self.X_train_flattened)
        self.X_test_scaled = self.scaler.transform(self.X_test_flattened)
        
        self.X_train_processed, self.y_train_processed = \
            process_data(self.X_train_scaled, self.timestep)
        self.X_test_processed, self.y_test_processed = \
            process_data(self.X_test_scaled, self.timestep)
        
    def train(self, verbose=0):
        history = self.model.fit(self.X_train_processed, self.y_train_processed,
                                 batch_size=self.batch_size, epochs=self.epochs)
        
        return history
    
    def evaluate(self):
        keras.evaluate(x=self.y_test_processed,
                       y=self.model.predict())

In [6]:
rnn = RNN(mt.time_series_data[0], epochs=2)

In [7]:
rnn.train(verbose=1)

Epoch 1/2
14609/14609 [==============================] - 138s 9ms/step - loss: 0.0198
Epoch 2/2
14609/14609 [==============================] - 129s 9ms/step - loss: 0.0075


In [8]:
mean_squared_error(rnn.scaler.inverse_transform(rnn.y_test_processed),
                   rnn.scaler.inverse_transform(
                       rnn.model.predict(rnn.X_test_processed)
                   ))

7.3311382219497885

In [9]:
for _ in rnn.scaler.inverse_transform(rnn.y_test_processed)-rnn.scaler.inverse_transform(rnn.model.predict(rnn.X_test_processed)):
    print(_)

[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.046718   -0.06804235  0.09577926  0.11521143 -0.10353851]
[-0.0467

[-0.01054799  0.04545919  0.01010879  0.15032667 -0.05657196]
[-0.00982034  0.04615585  0.01263221  0.15194505 -0.05593872]
[-0.00919187  0.04686157  0.01499351  0.15340132 -0.05575562]
[-0.00872361  0.0474786   0.01703246  0.15461439 -0.05583191]
[-0.00843465  0.04795448  0.01868232  0.15556329 -0.05603027]
[-0.00830972  0.04828064  0.01993926  0.15625948 -0.0562439 ]
[-0.00831544  0.04847042  0.02084811  0.15674585 -0.056427  ]
[-0.00841367  0.04855243  0.02146514  0.15706533 -1.0565567 ]
[-0.03723657  0.03156702 -0.00915925  0.13897985 -0.76853943]
[-0.05315816  0.01803295 -0.03183477  0.11818308 -0.50523376]
[-0.05767381  0.01337282 -0.04260747  0.10916704 -0.29254913]
[-0.05336701  0.01644365 -0.04188363  0.11062902 -0.14016724]
[-0.04371297  0.02471391 -0.03278463  0.11911577 -0.04462433]
[-0.03200471  0.0353903  -0.01922242  0.1308679   0.0046463 ]
[-0.02077424  0.04610245 -0.00474374  0.14279741  0.02065277]
[-0.0116018   0.05521957  0.00806602  0.15287203  0.01581573]
[-0.0051

[-0.09309328  0.1990618  -0.14880483  0.00948137 -0.81378174]
[-0.09314287  0.19903748 -0.14875524  0.00949472 -0.81378937]
[-0.09319627  0.19900744 -0.1487314   0.00950044 -0.81378937]
[-0.09324777  0.19897549 -0.14872567  0.00949567 -0.81377411]
[-0.09329259  0.19894736 -0.14872186  0.00949663 -0.81376648]
[-0.09332979  0.19892304 -0.14872186  0.00949377 -0.81375885]
[-0.09335935  0.19890682 -0.14872281  0.00949567 -0.81375885]
[-0.09338415  0.198893   -0.14871804  0.00950044 -0.81376648]
[-0.09340227  0.19888441 -0.14871423  0.00950426 -0.81376648]
[-0.09341466  0.19887964 -0.14870755  0.00950902 -0.81377411]
[-0.09342515  0.19887774 -0.14870183  0.00951665 -0.81378174]
[-0.09343374  0.19887583 -0.1486942   0.00952142 -0.81378937]
[-0.09344041  0.19887631 -0.14868562  0.00952714 -0.813797  ]
[-0.09344614  0.19887583 -0.14868085  0.00953096 -0.81380463]
[-0.09344995  0.19887631 -0.14867799  0.00953477 -0.81380463]
[-0.09345376  0.19887774 -0.14867418  0.00953859 -0.81380463]
[-0.0934

[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.84616315 -0.03440817 -0.92284983 -1.0177356  -6.52395391]
[-0.8461

[-0.55912387  0.0958461  -0.98149318 -0.94422728 -4.96905327]
[-0.55903852  0.09580891 -0.98166102 -0.94421011 -4.96903801]
[-0.55896366  0.09577648 -0.9818117  -0.94418722 -4.96902275]
[-0.55889261  0.09574597 -0.98195094 -0.94416338 -4.96900749]
[-0.55883205  0.09572165 -0.98206633 -0.94414335 -4.96899223]
[-0.55877578  0.09569781 -0.98217696 -0.94412619 -4.96896935]
[-0.55872476  0.09568016 -0.98227328 -0.94410998 -4.96896172]
[-0.55867851  0.09565918 -0.9823572  -0.94409186 -4.96895409]
[-0.55863464  0.09564488 -0.98243541 -0.94407755 -4.96893883]
[-0.55859649  0.09562771 -0.98250121 -0.94406229 -4.96892357]
[-0.55856168  0.09561865 -0.98256606 -0.94404703 -4.96891594]
[-0.5585345   0.09560578 -0.98262042 -0.94403845 -4.96890831]
[-0.55850637  0.09559624 -0.98266906 -0.9440251  -4.96890831]
[-0.55847919  0.0955867  -0.98271292 -0.94401079 -4.96890831]
[-0.55845726  0.0955786  -0.98275203 -0.94400698 -4.96889305]
[-0.55843818  0.09557335 -0.98278636 -0.94399744 -4.96888542]
[-0.5584

 -6.13467979e+00]
[-7.70363212e-01  6.09319948e-04 -9.64204013e-01 -9.88591254e-01
 -6.13467979e+00]
[-7.70363212e-01  6.09319948e-04 -9.64204013e-01 -9.88591254e-01
 -6.13467979e+00]
[-7.70363212e-01  6.09319948e-04 -9.64204013e-01 -9.88591254e-01
 -6.13467979e+00]
[-7.70363212e-01  6.09319948e-04 -9.64204013e-01 -9.88591254e-01
 -7.13467979e+00]
[-0.81787813 -0.00943955 -0.99980086 -1.01300824 -6.79807091]
[-0.82914245 -0.01894005 -0.99873751 -1.0365144  -6.5333004 ]
[-0.82932985 -0.02377375 -0.99048346 -1.04267132 -6.34648705]
[-0.82210386 -0.02380427 -0.97776431 -1.036852   -6.22962761]
[-0.81197774 -0.0205179  -0.96440428 -1.02477753 -6.16856194]
[-0.80202663 -0.01566466 -0.95283812 -1.01109993 -6.1466732 ]
[-0.79393613 -0.01064976 -0.94422168 -0.9988299  -6.14883995]
[-0.7883333  -0.00638207 -0.93877238 -0.98947245 -6.16314507]
[-7.85125136e-01 -3.28691653e-03 -9.36100185e-01 -9.83431876e-01
 -6.18147087e+00]
[-7.83859134e-01 -1.42582098e-03 -9.35573757e-01 -9.80367720e-01
 -6.19

In [10]:
for _ in enumerate(rnn.scaler.inverse_transform(rnn.y_test_processed[:100])): print(_)

(0, array([  0.,   0.,   0.,   0., 110.]))
(1, array([  0.,   0.,   0.,   0., 110.]))
(2, array([  0.,   0.,   0.,   0., 110.]))
(3, array([  0.,   0.,   0.,   0., 110.]))
(4, array([  0.,   0.,   0.,   0., 110.]))
(5, array([  0.,   0.,   0.,   0., 110.]))
(6, array([  0.,   0.,   0.,   0., 110.]))
(7, array([  0.,   0.,   0.,   0., 110.]))
(8, array([  0.,   0.,   0.,   0., 110.]))
(9, array([  0.,   0.,   0.,   0., 110.]))
(10, array([  0.,   0.,   0.,   0., 110.]))
(11, array([  0.,   0.,   0.,   0., 110.]))
(12, array([  0.,   0.,   0.,   0., 110.]))
(13, array([  0.,   0.,   0.,   0., 110.]))
(14, array([  0.,   0.,   0.,   0., 110.]))
(15, array([  0.,   0.,   0.,   0., 110.]))
(16, array([  0.,   0.,   0.,   0., 110.]))
(17, array([  0.,   0.,   0.,   0., 110.]))
(18, array([  0.,   0.,   0.,   0., 110.]))
(19, array([  0.,   0.,   0.,   0., 110.]))
(20, array([  0.,   0.,   0.,   0., 110.]))
(21, array([  0.,   0.,   0.,   0., 110.]))
(22, array([  0.,   0.,   0.,   0., 110.])

In [520]:
for _ in enumerate(rnn.scaler.inverse_transform(rnn.model.predict(rnn.X_test_processed[:100])).astype(np.int32)): print(_)

(0, array([  0,   0,   0,   0, 109], dtype=int32))
(1, array([  0,   0,   0,   0, 109], dtype=int32))
(2, array([  0,   0,   0,   0, 109], dtype=int32))
(3, array([  0,   0,   0,   0, 109], dtype=int32))
(4, array([  0,   0,   0,   0, 109], dtype=int32))
(5, array([  0,   0,   0,   0, 109], dtype=int32))
(6, array([  0,   0,   0,   0, 109], dtype=int32))
(7, array([  0,   0,   0,   0, 109], dtype=int32))
(8, array([  0,   0,   0,   0, 109], dtype=int32))
(9, array([  0,   0,   0,   0, 109], dtype=int32))
(10, array([  0,   0,   0,   0, 109], dtype=int32))
(11, array([  0,   0,   0,   0, 109], dtype=int32))
(12, array([  0,   0,   0,   0, 109], dtype=int32))
(13, array([  0,   0,   0,   0, 109], dtype=int32))
(14, array([  0,   0,   0,   0, 109], dtype=int32))
(15, array([  0,   0,   0,   0, 109], dtype=int32))
(16, array([  0,   0,   0,   0, 109], dtype=int32))
(17, array([  0,   0,   0,   0, 109], dtype=int32))
(18, array([  0,   0,   0,   0, 109], dtype=int32))
(19, array([  0,   0, 

In [171]:
mt = MidiTokenizer('data/no_quarter/NoQuarter.mid')

In [195]:
c = mt.time_series_data[5]
c = c.reshape(c.shape[0], c.shape[2], c.shape[1])

In [211]:
s1 = rnn.X_train_processed.shape
s2 = rnn.X_train_processed[0].shape

rnn.X_train_processed[0][0,0]

array([-1, -1], dtype=int32)